In [2]:
import os
from string import Template

In [5]:
SLURM_DIR = './SLURMS/Figure5/'
#os.makedirs(SLURM_DIR)

In [3]:
SLURM_TEMPLATE = Template('''#!/bin/bash
#SBATCH -p cox
#SBATCH -n 1 # Number of cores
#SBATCH -N 1 # Ensure that all cores are on one machine
#SBATCH --gres=gpu
#SBATCH --array=1-4
#SBATCH --mem=$memory
#SBATCH -t 3-00:00
#SBATCH --mail-type=ALL
#SBATCH --mail-user=ian.svetkey@gmail.com
#SBATCH -o /n/home05/isvetkey/cnn-stimuli/SLURMS/Figure5/LOGS/%A_%a.out
#SBATCH -e /n/home05/isvetkey/cnn-stimuli/SLURMS/Figure5/LOGS/%A_%a.err

# Print this sub-job's task ID
echo "My SLURM_ARRAY_TASK_ID: " $SLURM_ARRAY_TASK_ID

source new-modules.sh

module load Anaconda/5.0.1-fasrc01

cd /n/home05/isvetkey/cnn-stimuli/

source activate CP

python run_regression_isvetkey.py $experiment $dataset $classifier $noise $SLURM_ARRAY_TASK_ID 

# end
exit 0;
''')

In [ ]:
# Full table of numbers --> flags
# 0 = [False, False, False]
# 1 = [False, False, True]
# 2 = [False, True, False]
# 3 = [False, True, True]
# 4 = [True, False, False]
# 5 = [True, False, True]
# 6 = [True, True, False]
# 7 = [True, True, True]

In [6]:
Experiments = {
  #ignore flag 1 for pos bc it's irrelevant: combos are [2, 3], [2], [3], and [] (or FTT, FTF, FFT and FFF)
  's.Figure5.position_common_scale': range(4), 
  's.Figure5.position_non_aligned_scale': range(4),
  #don't ignore flag 1 for others: all combos from [] to [1, 2, 3] (or FFF, FFT, FTF, FTT, TFF, etc to TTT)
  's.Figure5.length': range(8),
  's.Figure5.direction': range(8),
  's.Figure5.angle': range(8), 
  's.Figure5.area': range(8),
  's.Figure5.volume': range(8),
  's.Figure5.curvature': range(8),
}

#are those numbers the amount of memory each one takes up?
Classifiers = {
  'MLP': '8000',
  'LeNet': '8000',
  'VGG19': '20000',
  'XCEPTION': '32000'
}

Noise = ['True', 'False']

for e in Experiments:
  Datasets = Experiments[e]
  for d in Datasets:
    for c in Classifiers:
      m = Classifiers[c]
      for n in Noise:
        
        new_slurm = SLURM_TEMPLATE.substitute(experiment=e, memory=m, dataset=str(d), classifier=c, noise=n, SLURM_ARRAY_TASK_ID='$SLURM_ARRAY_TASK_ID')
        with open(os.path.join(SLURM_DIR, e+'_'+str(d)+'_'+c+'_'+n+'.sbatch'), 'w') as f:
          f.write(new_slurm)

